# Импорт библиотек

In [115]:
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [116]:
dir = os.path.abspath(os.curdir)
dir

'c:\\Users\\Anton\\Desktop\\python\\task'

# Подготовка данных

In [117]:
# reading json into DF
orders = pd.read_json('trial_task.json', encoding="utf-8")

def unpack(el: list) -> "pd.DataFrame":
    """ Unpacking "products" and joining them for order_id

    Args:
        el (list): List of json elements

    Returns:
        pd.DataFrame: DataFrame with products list as columns
    """
    order_id, warehouse_name, highway_cost, produts = el
    left_df = pd.DataFrame([(order_id, warehouse_name, highway_cost)],
                            columns=['order_id', 'warehouse_name', 'highway_cost'])

    return left_df.join(pd.DataFrame.from_dict(produts), how='cross')

res_df = pd.DataFrame()

# joining all order_ids together in one DF
for el in orders.values:
    res_df = pd.concat([res_df, unpack(el)])

res_df.reset_index(inplace=True, drop=True)

In [118]:
res_df

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,-70,ломтик июльского неба,450,1
1,11973,Мордор,-70,билет в Израиль,1000,3
2,11973,Мордор,-70,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1
4,85794,отель Лето,-50,зеленая пластинка,10,2
...,...,...,...,...,...,...
192,79293,отель Лето,-75,автограф Стаса Барецкого,600,1
193,79293,отель Лето,-75,ломтик июльского неба,450,1
194,2930,Мордор,-30,плюмбус,250,2
195,2930,Мордор,-30,плюмбус,250,1


# Задача 1

In [119]:

def get_warehouse_cost(df: 'pd.DataFrame') -> 'pd.DataFrame':
    warehouse_order = df[df['order_id'] == df['order_id'].unique()[0]]
    return warehouse_order['highway_cost'].unique()[0]/warehouse_order['quantity'].sum()

warehouse_cost = pd.DataFrame(res_df.groupby(['warehouse_name']).apply(get_warehouse_cost), columns=['warehouse_cost']).reset_index()
warehouse_cost

,warehouse_name,warehouse_cost
0,Мордор,-10.0
1,гиперборея,-20.0
2,остров невезения,-5.0
3,отель Лето,-25.0
4,хутор близ Диканьки,-15.0


# Задача 2

In [120]:
res_df_cost = res_df.set_index('warehouse_name').join(warehouse_cost.set_index('warehouse_name'))
res_df_cost['expense'] = res_df_cost['warehouse_cost'] * res_df_cost['quantity']
res_df_cost.reset_index(inplace=True)
res_df_cost

,warehouse_name,order_id,highway_cost,product,price,quantity,warehouse_cost,expense
0,Мордор,11973,-70,ломтик июльского неба,450,1,-10.0,-10.0
1,Мордор,11973,-70,билет в Израиль,1000,3,-10.0,-30.0
2,Мордор,11973,-70,статуэтка Ленина,200,3,-10.0,-30.0
3,Мордор,33684,-30,билет в Израиль,1000,2,-10.0,-20.0
4,Мордор,33684,-30,зеленая пластинка,10,1,-10.0,-10.0
...,...,...,...,...,...,...,...,...
192,хутор близ Диканьки,15237,-60,плюмбус,250,1,-15.0,-15.0
193,хутор близ Диканьки,15237,-60,зеленая пластинка,10,2,-15.0,-30.0
194,хутор близ Диканьки,28106,-30,подписка на suppi-блог,150,1,-15.0,-15.0
195,хутор близ Диканьки,28106,-30,статуэтка Ленина,200,1,-15.0,-15.0


In [121]:
product_sum = res_df_cost.groupby(['product']).sum().reset_index()
product_sum['profit'] = product_sum['price'] - product_sum['expense']
product_sum[['product','quantity', 'price', 'expense', 'profit']]

,product,quantity,price,expense,profit
0,автограф Стаса Барецкого,48,16200,-820.0,17020.0
1,билет в Израиль,58,29000,-1050.0,30050.0
2,зеленая пластинка,61,310,-920.0,1230.0
3,ломтик июльского неба,42,9450,-670.0,10120.0
4,плюмбус,65,8250,-940.0,9190.0
5,подписка на suppi-блог,33,2850,-545.0,3395.0
6,статуэтка Ленина,68,7400,-935.0,8335.0


# Задача 3

In [122]:
res_df_cost['order_price'] = res_df_cost['price']*res_df_cost['quantity']
order_profit_df = res_df_cost.groupby(['order_id', 'highway_cost']).sum().reset_index()
order_profit_df['order_profit'] = order_profit_df['order_price'] + order_profit_df['highway_cost']

print("Средняя прибыль заказов", order_profit_df['order_profit'].mean())
order_profit_df[['order_id', 'order_profit']]

Средняя прибыль заказов 1352.3


,order_id,order_profit
0,124,705
1,1391,490
2,2091,1300
3,2108,200
4,2558,355
...,...,...
95,98100,1590
96,98423,1170
97,99220,1075
98,99246,1755


# Задача 4

In [123]:
# group by  warehouse_name and product
warehouse_profit = res_df_cost.groupby(
    ['warehouse_name', 'product']).sum().reset_index()
# calculating product profit inside warehouse
warehouse_profit['profit'] = warehouse_profit['order_price'] + warehouse_profit['expense']

# DF with warehouse_profit, calculating warehouse profit inside warehouse
warehouse_profit = warehouse_profit.set_index('warehouse_name').join(
    warehouse_profit.groupby(['warehouse_name']).sum().rename(
        columns={'profit':'profit_warehouse'})['profit_warehouse']).reset_index()

# calculating percent_profit_product_of_warehouse
warehouse_profit['percent_profit_product_of_warehouse'] = (
    warehouse_profit['profit']/warehouse_profit['profit_warehouse']) * 100
# resulting DF 
warehouse_profit = warehouse_profit[['warehouse_name',
                                      'product',
                                        'quantity',
                                          'profit',
                                            'percent_profit_product_of_warehouse']]
warehouse_profit

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2360.0,13.841642
1,Мордор,билет в Израиль,9,8910.0,52.258065
2,Мордор,зеленая пластинка,11,0.0,0.000000
3,Мордор,ломтик июльского неба,3,1320.0,7.741935
4,Мордор,плюмбус,6,1440.0,8.445748
5,Мордор,подписка на suppi-блог,8,1120.0,6.568915
6,Мордор,статуэтка Ленина,10,1900.0,11.143695
7,гиперборея,автограф Стаса Барецкого,12,6960.0,18.181818
8,гиперборея,билет в Израиль,21,20580.0,53.761755
9,гиперборея,зеленая пластинка,10,-100.0,-0.261233


# Задача 5

In [124]:
warehouse_profit = warehouse_profit.sort_values(
    by=['warehouse_name',
        'percent_profit_product_of_warehouse'],
          ascending=False).reset_index(drop=True)
x = 0
accumulated = []
warehouse_profit['accumulated_percent_profit_product_of_warehouse'] = warehouse_profit['percent_profit_product_of_warehouse']
# creating list of accumulating percents
for el in  warehouse_profit['accumulated_percent_profit_product_of_warehouse'].values:
  x += el
  accumulated.append(x)
  # going arround python round errors
  if (x % 100 < 0.001) | (x % 100 > 99.999) :
    x = 0
warehouse_profit['accumulated_percent_profit_product_of_warehouse'] = accumulated 
warehouse_profit

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,хутор близ Диканьки,автограф Стаса Барецкого,22,12870.0,34.559613,34.559613
1,хутор близ Диканьки,билет в Израиль,10,9850.0,26.450054,61.009667
2,хутор близ Диканьки,плюмбус,27,6345.0,17.038131,78.047798
3,хутор близ Диканьки,статуэтка Ленина,21,3885.0,10.432331,88.480129
4,хутор близ Диканьки,ломтик июльского неба,7,3045.0,8.176692,96.656821
5,хутор близ Диканьки,подписка на suppi-блог,10,1350.0,3.625134,100.281955
6,хутор близ Диканьки,зеленая пластинка,21,-105.0,-0.281955,100.000000
7,отель Лето,билет в Израиль,15,14625.0,54.066543,54.066543
8,отель Лето,автограф Стаса Барецкого,8,4600.0,17.005545,71.072089
9,отель Лето,ломтик июльского неба,9,3825.0,14.140481,85.212569


# Задача 6

In [125]:
# 
warehouse_profit['category'] = 'B'
warehouse_profit.loc[warehouse_profit[warehouse_profit.accumulated_percent_profit_product_of_warehouse <= 70].index, 'category'] = 'A'
warehouse_profit.loc[warehouse_profit[warehouse_profit.accumulated_percent_profit_product_of_warehouse > 90].index, 'category'] = 'C'
warehouse_profit

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,хутор близ Диканьки,автограф Стаса Барецкого,22,12870.0,34.559613,34.559613,A
1,хутор близ Диканьки,билет в Израиль,10,9850.0,26.450054,61.009667,A
2,хутор близ Диканьки,плюмбус,27,6345.0,17.038131,78.047798,B
3,хутор близ Диканьки,статуэтка Ленина,21,3885.0,10.432331,88.480129,B
4,хутор близ Диканьки,ломтик июльского неба,7,3045.0,8.176692,96.656821,C
5,хутор близ Диканьки,подписка на suppi-блог,10,1350.0,3.625134,100.281955,C
6,хутор близ Диканьки,зеленая пластинка,21,-105.0,-0.281955,100.000000,C
7,отель Лето,билет в Израиль,15,14625.0,54.066543,54.066543,A
8,отель Лето,автограф Стаса Барецкого,8,4600.0,17.005545,71.072089,B
9,отель Лето,ломтик июльского неба,9,3825.0,14.140481,85.212569,B
